In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
import tensorflow_datasets as tfds

## Load Titanic Dataset

In [69]:
data = tfds.load('titanic')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1309 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/titanic/incomplete.39BCO9_4.0.0/titanic-train.tfrecord*...:   0%|         …

Dataset titanic downloaded and prepared to /root/tensorflow_datasets/titanic/4.0.0. Subsequent calls will reuse this data.


In [70]:
train_data = data['train']

In [78]:
df = tfds.as_dataframe(train_data)

In [79]:
df.head()

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,survived,ticket
0,30.0,b'Unknown',-1,b'Unknown',2,13.0000,"b'Sarnia, ON'","b'McCrie, Mr. James Matthew'",0,1,0,0,0,b'233478'
1,37.0,b'Unknown',98,b'Unknown',2,7.9250,"b'Ruotsinphytaa, Finland New York, NY'","b'Gustafsson, Mr. Anders Vilhelm'",0,2,0,2,0,b'3101276'
2,28.0,b'9',-1,b'Unknown',2,13.0000,b'Spain',"b'Reynaldo, Ms. Encarnacion'",0,1,1,0,1,b'230434'
3,18.0,b'Unknown',-1,b'Unknown',2,73.5000,"b'Lyndhurst, England'","b'Davies, Mr. Charles Henry'",0,1,0,0,0,b'S.O.C. 14879'
4,-1.0,b'Unknown',-1,b'Unknown',0,7.8958,b'Unknown',"b'Gheorgheff, Mr. Stanio'",0,2,0,0,0,b'349254'


In [80]:
df.info()

<class 'tensorflow_datasets.core.as_dataframe.as_dataframe.<locals>.StyledDataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        1309 non-null   float32
 1   boat       1309 non-null   object 
 2   body       1309 non-null   int32  
 3   cabin      1309 non-null   object 
 4   embarked   1309 non-null   int64  
 5   fare       1309 non-null   float32
 6   home.dest  1309 non-null   object 
 7   name       1309 non-null   object 
 8   parch      1309 non-null   int32  
 9   pclass     1309 non-null   int64  
 10  sex        1309 non-null   int64  
 11  sibsp      1309 non-null   int32  
 12  survived   1309 non-null   int64  
 13  ticket     1309 non-null   object 
dtypes: float32(2), int32(3), int64(4), object(5)
memory usage: 117.7+ KB


## Preprocessing Data

In [81]:
df = df.drop(columns=['boat', 'cabin', 'home.dest','name', 'ticket','fare', 'parch','embarked','sibsp', 'body'])

In [82]:
df.head()

,age,pclass,sex,survived
0,30.0,1,0,0
1,37.0,2,0,0
2,28.0,1,1,1
3,18.0,1,0,0
4,-1.0,2,0,0


In [83]:
df['age'] = df['age'].astype('int64')

In [85]:
X = df[['age', 'pclass', 'sex']]
Y = df['survived']

In [86]:
features = np.vstack(X.to_numpy())

In [87]:
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(features)
X_norm = norm_l(features)
Y = Y.to_numpy()

In [88]:
print('Normalization result: ', X_norm[:5])

Normalization result:  tf.Tensor(
[[ 0.35507897 -0.35209072 -0.7434969 ]
 [ 0.74698323  0.8419164  -0.7434969 ]
 [ 0.24310634 -0.35209072  1.3449954 ]
 [-0.31675687 -0.35209072 -0.7434969 ]
 [-1.380497    0.8419164  -0.7434969 ]], shape=(5, 3), dtype=float32)


In [89]:
print(X_norm.shape, Y.shape)

(1309, 3) (1309,)


In [90]:
Y = Y.reshape(-1, 1)

In [91]:
print(X_norm.shape, Y.shape)

(1309, 3) (1309, 1)


In [92]:
Xt = np.tile(X_norm, (1000, 1))
Yt = np.tile(Y, (1000, 1))

In [93]:
print(Xt.shape, Yt.shape)

(1309000, 3) (1309000, 1)


## NN Using Tensorflow

In [94]:
tf.random.set_seed(1234)
model = tf.keras.Sequential([
    tf.keras.Input(shape=(3,)),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [95]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

In [96]:
history = model.fit(Xt, Yt, epochs=10)

Epoch 1/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 0.4533
Epoch 2/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 86s 2ms/step - loss: 0.4266
Epoch 3/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 67s 2ms/step - loss: 0.4127
Epoch 4/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - loss: 0.4084
Epoch 5/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.4059
Epoch 6/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - loss: 0.4040
Epoch 7/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - loss: 0.4021
Epoch 8/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 0.4011
Epoch 9/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 64s 2ms/step - loss: 0.4008
Epoch 10/10
40907/40907 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - loss: 0.3995


In [97]:
X_test = [30.0, 1.0, 0.0]
Xn = norm_l(X_test)
prediction = model.predict(Xn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


In [99]:
if (prediction >= 0.5):
    print('Survive')
else:
    print('Die')

Die
